# Developer
executorlib is designed to work out of the box for up-scaling Python functions and distribute them on a high performance computing (HPC) cluster. Most users should only import the `Executor` class from executorlib and should not need to use any of the internal functionality covered in this section. Still for more advanced applications beyond the submission of Python functions executorlib provides additional functionality. The functionality in this section is not officially supported and might change in future versions without further notice. 

## Communication
The key functionality of the `executorlib` package is the up-scaling of python functions with thread based parallelism, 
MPI based parallelism or by assigning GPUs to individual python functions. In the background this is realized using a 
combination of the [zero message queue](https://zeromq.org) and [cloudpickle](https://github.com/cloudpipe/cloudpickle) 
to communicate binary python objects. The `executorlib.standalone.interactive.communication.SocketInterface` is an abstraction of this 
interface, which is used in the other classes inside `executorlib` and might also be helpful for other projects. It 
comes with a series of utility functions:

* `executorlib.standalone.interactive.communication.interface_bootup()`: To initialize the interface
* `executorlib.standalone.interactive.communication.interface_connect()`: To connect the interface to another instance
* `executorlib.standalone.interactive.communication.interface_send()`: To send messages via this interface 
* `executorlib.standalone.interactive.communication.interface_receive()`: To receive messages via this interface 
* `executorlib.standalone.interactive.communication.interface_shutdown()`: To shutdown the interface

While `executorlib` was initially designed for up-scaling python functions for HPC, the same functionality can be 
leveraged to up-scale any executable independent of the programming language it is developed in.

## External Executables
On extension beyond the submission of Python functions is the communication with an external executable. This could be any kind of program written in any programming language which does not provide Python bindings so it cannot be represented in Python functions. 

### Subprocess
If the external executable is called only once, then the call to the external executable can be represented in a Python function with the [subprocess](https://docs.python.org/3/library/subprocess.html) module of the Python standard library. In the example below the shell command `echo test` is submitted to the `execute_shell_command()` function, which itself is submitted to the `Executor` class.

In [1]:
from executorlib import Executor

In [2]:
def execute_shell_command(
    command: list, universal_newlines: bool = True, shell: bool = False
):
    import subprocess

    return subprocess.check_output(
        command, universal_newlines=universal_newlines, shell=shell
    )

In [3]:
with Executor(backend="local") as exe:
    future = exe.submit(
        execute_shell_command,
        ["echo", "test"],
        universal_newlines=True,
        shell=False,
    )
    print(future.result())

test



### Interactive
The more complex case is the interaction with an external executable during the run time of the executable. This can be implemented with executorlib using the block allocation `block_allocation=True` feature. The external executable is started as part of the initialization function `init_function` and then the indivdual functions submitted to the `Executor` class interact with the process which is connected to the external executable. 

Starting with the definition of the executable, in this example it is a simple script which just increases a counter. The script is written in the file `count.py` so it behaves like an external executable, which could also use any other progamming language. 

In [4]:
count_script = """\
def count(iterations):
    for i in range(int(iterations)):
        print(i)
    print("done")


if __name__ == "__main__":
    while True:
        user_input = input()
        if "shutdown" in user_input:
            break
        else:
            count(iterations=int(user_input))
"""

with open("count.py", "w") as f:
    f.writelines(count_script)

The connection to the external executable is established in the initialization function `init_function` of the `Executor` class. By using the [subprocess](https://docs.python.org/3/library/subprocess.html) module from the standard library two process pipes are created to communicate with the external executable. One process pipe is connected to the standard input `stdin` and the other is connected to the standard output `stdout`. 

In [5]:
def init_process():
    import subprocess

    return {
        "process": subprocess.Popen(
            ["python", "count.py"],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            universal_newlines=True,
            shell=False,
        )
    }

The interaction function handles the data conversion from the Python datatypes to the strings which can be communicated to the external executable. It is important to always add a new line `\n` to each command send via the standard input `stdin` to the external executable and afterwards flush the pipe by calling `flush()` on the standard input pipe `stdin`.  

In [6]:
def interact(shell_input, process, lines_to_read=None, stop_read_pattern=None):
    process.stdin.write(shell_input)
    process.stdin.flush()
    lines_count = 0
    output = ""
    while True:
        output_current = process.stdout.readline()
        output += output_current
        lines_count += 1
        if stop_read_pattern is not None and stop_read_pattern in output_current:
            break
        elif lines_to_read is not None and lines_to_read == lines_count:
            break
    return output

Finally, to close the process after the external executable is no longer required it is recommended to define a shutdown function, which communicates to the external executable that it should shutdown. In the case of the `count.py` script defined above this is achieved by sending the keyword `shutdown`. 

In [7]:
def shutdown(process):
    process.stdin.write("shutdown\n")
    process.stdin.flush()

With these utility functions is to possible to communicate with any kind of external executable. Still for the specific implementation of the external executable it might be necessary to adjust the corresponding Python functions. Therefore this functionality is currently limited to developers and not considered a general feature of executorlib. 

In [8]:
with Executor(
    max_workers=1,
    init_function=init_process,
    block_allocation=True,
) as exe:
    future = exe.submit(
        interact, shell_input="4\n", lines_to_read=5, stop_read_pattern=None
    )
    print(future.result())
    future_shutdown = exe.submit(shutdown)
    print(future_shutdown.result())

0
1
2
3
done

None


## License
```
BSD 3-Clause License

Copyright (c) 2022, Jan Janssen
All rights reserved.

Redistribution and use in source and binary forms, with or without
modification, are permitted provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, this
  list of conditions and the following disclaimer.

* Redistributions in binary form must reproduce the above copyright notice,
  this list of conditions and the following disclaimer in the documentation
  and/or other materials provided with the distribution.

* Neither the name of the copyright holder nor the names of its
  contributors may be used to endorse or promote products derived from
  this software without specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
```

## Modules
While it is not recommended to link to specific internal components of executorlib in external Python packages but rather only the `Executor` class should be used as central interface to executorlib, the internal architecture is briefly outlined below. 
* `backend` - the backend module contains the functionality for the Python processes created by executorlib to execute the submitted Python functions.
* `base` - the base module contains the definition of the executorlib `ExecutorBase` class which is internally used to create the different interfaces. To compare if an given `Executor` class is based on executorlib compare with the `ExecutorBase` class which can be imported as `from executorlib.base.executor import ExecutorBase`.
* `cache` - the cache module defines the file based communication for the [HPC submission mode]().
* `interactive` - the interactive modules defines the [zero message queue](https://zeromq.org) based communication for the [local mode]() and the [HPC allocation mode]().
* `standalone` - the standalone module contains a number of utility functions which only depend on external libraries and do not have any internal dependency to other parts of `executorlib`. This includes the functionality to generate executable commands, the [h5py](https://www.h5py.org) based interface for caching, a number of input checks, routines to plot the dependencies of a number of future objects, functionality to interact with the [queues defined in the Python standard library](https://docs.python.org/3/library/queue.html), the interface for serialization based on [cloudpickle](https://github.com/cloudpipe/cloudpickle) and finally an extension to the [threading](https://docs.python.org/3/library/threading.html) of the Python standard library.

Given the level of separation the integration of submodules from the standalone module in external software packages should be the easiest way to benefit from the developments in executorlib beyond just using the `Executor` class. 